In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import explained_variance_score


In [2]:
path_dataset = "../hfactory_magic_folders/hi__paris_hackaton/building_energy_efficiency/dataset/train"
path_x = "{}/train_features_sent.csv".format(path_dataset)
path_y = "{}/train_labels_sent.csv".format(path_dataset)

In [60]:
X = pd.read_csv(path_x)
Y = pd.read_csv(path_y)

/home/frank/anaconda3/envs/BCG/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
# X = X.iloc[:1000, :]
# Y = Y.iloc[:1000, :]

In [61]:
print(len(X))
print(len(Y))

1010684
1010684


In [85]:
# df_datatypes = pd.DataFrame(X.dtypes)
df_non_null_count = X.count()

In [97]:
df_non_null_count = X.count()
print("Total number of columns: {}".format(len(df_non_null_count)))

col_valid = []
threshold = 200000

for col, non_null in df_non_null_count.iteritems():
    # print(col, non_null)
    if non_null > threshold:
        col_valid.append(col)

print("Total number of columns: {}".format(len(col_valid)))

64


In [88]:
# Types and non null features
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010684 entries, 0 to 1010683
Data columns (total 71 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   level_0                           1010684 non-null  int64  
 1   additional_heat_generators        1010684 non-null  object 
 2   additional_water_heaters          209994 non-null   object 
 3   altitude                          992545 non-null   float64
 4   area_code                         1010684 non-null  int64  
 5   balcony_depth                     105791 non-null   object 
 6   bearing_wall_material             956635 non-null   object 
 7   building_category                 1010684 non-null  object 
 8   building_class                    1010684 non-null  object 
 9   building_height_ft                994468 non-null   float64
 10  building_period                   1010684 non-null  object 
 11  building_total_area_sqft          949

In [81]:
info = X.info()
info.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010684 entries, 0 to 1010683
Data columns (total 71 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   level_0                           1010684 non-null  int64  
 1   additional_heat_generators        1010684 non-null  object 
 2   additional_water_heaters          209994 non-null   object 
 3   altitude                          992545 non-null   float64
 4   area_code                         1010684 non-null  int64  
 5   balcony_depth                     105791 non-null   object 
 6   bearing_wall_material             956635 non-null   object 
 7   building_category                 1010684 non-null  object 
 8   building_class                    1010684 non-null  object 
 9   building_height_ft                994468 non-null   float64
 10  building_period                   1010684 non-null  object 
 11  building_total_area_sqft          949

AttributeError: 'NoneType' object has no attribute 'columns'

In [77]:
print(type(X.info()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010684 entries, 0 to 1010683
Data columns (total 71 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   level_0                           1010684 non-null  int64  
 1   additional_heat_generators        1010684 non-null  object 
 2   additional_water_heaters          209994 non-null   object 
 3   altitude                          992545 non-null   float64
 4   area_code                         1010684 non-null  int64  
 5   balcony_depth                     105791 non-null   object 
 6   bearing_wall_material             956635 non-null   object 
 7   building_category                 1010684 non-null  object 
 8   building_class                    1010684 non-null  object 
 9   building_height_ft                994468 non-null   float64
 10  building_period                   1010684 non-null  object 
 11  building_total_area_sqft          949

In [73]:
cat_cols = X.select_dtypes(include="object").columns.to_list()
print("Category columns: ", cat_cols)
print("Number of Category columns: ", len(cat_cols))

Category columns:  ['additional_heat_generators', 'additional_water_heaters', 'balcony_depth', 'bearing_wall_material', 'building_category', 'building_class', 'building_period', 'building_type', 'building_use_type_description', 'clay_risk_level', 'consumption_measurement_date', 'has_balcony', 'heat_generators', 'heating_energy_source', 'heating_type', 'is_crossing_building', 'lower_floor_adjacency_type', 'lower_floor_insulation_type', 'lower_floor_material', 'main_heat_generators', 'main_heating_type', 'main_water_heaters', 'main_water_heating_type', 'outer_wall_materials', 'outer_wall_thickness', 'radon_risk_level', 'renewable_energy_sources', 'roof_material', 'thermal_inertia', 'upper_floor_adjacency_type', 'upper_floor_insulation_type', 'upper_floor_material', 'ventilation_type', 'wall_insulation_type', 'water_heaters', 'water_heating_energy_source', 'water_heating_type', 'window_filling_type', 'window_frame_material', 'window_glazing_type', 'window_orientation']
Number of Category 

In [ ]:
cols_to_drop = []

In [63]:
X.head()

,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,...,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity
0,111323,[wood stove or insert],NaN,328.08400,881,NaN,CONCRETE,[],[],13.779528,...,internal,[indeterminate oil boiler],oil,individual,argon or krypton,pvc,double glazing,1.116894,"[est,west]",15.347292
1,265182,[],NaN,360.89240,260,NaN,NaN,[condo],[2 to 11],43.635172,...,internal,[indeterminate gas boiler],gas,individual,argon or krypton,wood,double glazing,1.133072,"[est,west,sud]",17.512631
2,927507,[],NaN,324.80316,573,NaN,WOOD - OTHER,[],[],19.685040,...,internal,[standard gas boiler],gas,individual,dry air,pvc,double glazing,1.084204,"[est,west]",16.779108
3,874620,[],NaN,288.71392,204,NaN,BRICKS,[],[],13.779528,...,internal,[electric storage tank],electricity,individual,NaN,metal with thermal break,triple glazing,0.983338,"[est,north,sud]",12.630033
4,563274,[],NaN,849.73756,374,NaN,GRITSTONE,[individual house],[individual],19.028872,...,internal,[thermodynamic electric hot water (pac or tank)],thermodynamic electric hot water (heat pump or...,individual,dry air,pvc,double glazing,1.084204,"[north,sud]",13.290529


In [64]:
Y.head()

,level_0,energy_consumption_per_annum
0,111323,290.166345
1,265182,169.818197
2,927507,144.499711
3,874620,158.742715
4,563274,82.223648


In [65]:
# X.drop(["level_0"], axis=1)
# Y = Y.drop(["level_0"], axis=1)

In [66]:
df = pd.merge(X, Y, how="inner", on="level_0")

In [67]:
len(df)

1010684

In [68]:
df = df.drop(["level_0"], axis=1)

In [69]:
len(df)

1010684

In [71]:
df.dropna(inplace=True)

In [72]:
len(df)

0

In [15]:
X_filter = df.drop(["energy_consumption_per_annum"], axis=1)

In [16]:
Y_filter = df["energy_consumption_per_annum"]

In [17]:
cat_cols = X.select_dtypes(include="object").columns.to_list()
print("Category columns: ", cat_cols)
print("Number of Category columns: ", len(cat_cols))

Category columns:  ['additional_heat_generators', 'additional_water_heaters', 'balcony_depth', 'bearing_wall_material', 'building_category', 'building_class', 'building_period', 'building_type', 'building_use_type_description', 'clay_risk_level', 'consumption_measurement_date', 'has_balcony', 'heat_generators', 'heating_energy_source', 'heating_type', 'is_crossing_building', 'lower_floor_adjacency_type', 'lower_floor_insulation_type', 'lower_floor_material', 'main_heat_generators', 'main_heating_type', 'main_water_heaters', 'main_water_heating_type', 'outer_wall_materials', 'outer_wall_thickness', 'radon_risk_level', 'renewable_energy_sources', 'roof_material', 'thermal_inertia', 'upper_floor_adjacency_type', 'upper_floor_insulation_type', 'upper_floor_material', 'ventilation_type', 'wall_insulation_type', 'water_heaters', 'water_heating_energy_source', 'water_heating_type', 'window_filling_type', 'window_frame_material', 'window_glazing_type', 'window_orientation']
Number of Category 

In [27]:
X[cat_cols].nunique()

additional_heat_generators        12
additional_water_heaters           9
balcony_depth                      4
bearing_wall_material             28
building_category                 13
building_class                    20
building_period                    7
building_type                      3
building_use_type_description      3
clay_risk_level                    3
consumption_measurement_date     793
has_balcony                        2
heat_generators                   98
heating_energy_source             16
heating_type                       2
is_crossing_building               6
lower_floor_adjacency_type         6
lower_floor_insulation_type        5
lower_floor_material              11
main_heat_generators              17
main_heating_type                159
main_water_heaters                14
main_water_heating_type          159
outer_wall_materials              16
outer_wall_thickness              28
radon_risk_level                   3
renewable_energy_sources           2
r

In [33]:
X["building_class"].unique()

array(['[]', '[2 to 11]', '[individual]', '[individual,2 to 11]', '[12+]',
       '[12+,individual,12+,12+,12+,individual,12+,12+,12+,12+]',
       '[12+,individual]',
       '[individual,individual,individual,individual,individual,individual,individual,individual,individual,individual]',
       '[individual,individual]', '[2 to 11,12+,individual]',
       '[2 to 11,12+]', '[2 to 11,individual]', '[12+,12+]',
       '[individual,2 to 11,individual,individual,individual,individual,individual,individual]',
       '[2 to 11,2 to 11,2 to 11,2 to 11]',
       '[individual,individual,individual]', '[12+,2 to 11,individual]',
       '[2 to 11,2 to 11]', '[individual,2 to 11,individual]',
       '[12+,2 to 11]'], dtype=object)

In [36]:
X[["building_class", "building_type"]]

,building_class,building_type
0,[],House
1,[2 to 11],Flat
2,[],House
3,[],House
4,[individual],House
...,...,...
995,[],Flat
996,[2 to 11],Flat
997,[individual],House
998,[individual],House


In [20]:
Y

,level_0,energy_consumption_per_annum
0,111323,290.166345
1,265182,169.818197
2,927507,144.499711
3,874620,158.742715
4,563274,82.223648
...,...,...
995,341234,250.836513
996,731113,331.302363
997,1277064,278.484614
998,213710,171.864307


In [24]:
len(Y)

1000

In [23]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=2,
                           learning_rate=1,
                           depth=2)
for col in cat_cols:
    X[col] = X[col].apply(str)

# Fit model
model.fit(X, Y, cat_features=cat_cols)

0:	learn: 86.5858258	total: 47.6ms	remaining: 47.6ms
1:	learn: 81.2327477	total: 48.9ms	remaining: 0us


In [ ]:
# Get predicted classes
# preds_class = model.predict(eval_data)